In [3]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.multicovariate_models.GroupsModelWrapper import GroupsModelWrapper
from src.multicovariate_models.DataLoader import DataLoader

In [19]:
%%time

base_dir = '../../'
df = pd.read_csv(os.path.join(base_dir, "data/dataframes/simulations-step0.01.csv"))
df = df.drop(columns=["origin_file"]).drop_duplicates()
"""main_df = main_df[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak']].round(3).drop_duplicates()"""
df['leak_amount'] = df['leak_amount'].str[:-3].astype(float)

# drop all the `dpressure` columns
dp_col_names = []
for col_name in df.columns:
    if col_name.endswith('_dpressure'):
        dp_col_names.append(col_name)

df = df.drop(dp_col_names, axis=1)
display(df)

,Sensor1,Sensor2,Sensor3,Sensor4,J-Apollo,J-RN2,J-RN1,node_with_leak,leak_amount
0,14.0689,14.3960,17.4695,19.1754,15.4271,15.9280,19.8981,Node_275-A,19.500
1,14.0689,14.3958,17.4695,19.1754,15.4271,15.9280,19.8981,Node_275-A,19.501
2,14.0688,14.3957,17.4695,19.1754,15.4271,15.9280,19.8981,Node_275-A,19.502
3,14.0688,14.3956,17.4694,19.1754,15.4271,15.9280,19.8981,Node_275-A,19.503
4,14.0687,14.3955,17.4694,19.1754,15.4271,15.9280,19.8981,Node_275-A,19.504
...,...,...,...,...,...,...,...,...,...
1009579,14.6058,15.6646,17.7761,19.2672,15.4271,15.9421,19.8981,Node_Jonctiune-267,7.696
1009580,14.6057,15.6646,17.7761,19.2672,15.4271,15.9421,19.8981,Node_Jonctiune-267,7.697
1009581,14.6057,15.6645,17.7760,19.2672,15.4271,15.9421,19.8981,Node_Jonctiune-267,7.698
1009582,14.6057,15.6645,17.7760,19.2672,15.4271,15.9421,19.8981,Node_Jonctiune-267,7.699


CPU times: user 3.3 s, sys: 3.26 s, total: 6.57 s
Wall time: 6.57 s


In [20]:
%%time

# prepare the data for all models
data_loader = DataLoader(df)
X_train, X_test, y_train, y_test, enc_node_dict = data_loader.get_random_data_split_by_node()
#print(X_train, X_test, y_train, y_test, enc_node_dict)

Train data shape: (907200, 10), test data shape: (100944, 10)
x_train shape: (907200, 7), y_train shape: (907200,)
CPU times: user 2.88 s, sys: 708 ms, total: 3.59 s
Wall time: 3.59 s


In [25]:
import numpy as np
import sklearn

n_train = X_train.shape[0]
n_test = X_test.shape[0]

X_train_preproc = data_loader.preprocess_abs_pressure(X_train)
X_test_preproc = data_loader.preprocess_abs_pressure(X_test)
print(X_train_preproc.sum(axis=1))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
for n_neighbours in range(1, 26):
    classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbours, p=np.inf)
    classifier.fit(X_train_preproc, y_train)

    y_hat = classifier.predict(X_test_preproc)

    total_diff = np.sum(np.abs(y_test - y_hat) > 0)
    ca = 1.0 - total_diff / n_test

    print(f'n_neighbours: {n_neighbours}, CA: {ca}')

n_neighbours: 1, CA: 0.5916844983357109
n_neighbours: 2, CA: 0.6208095577746078
n_neighbours: 3, CA: 0.5891088128071009
n_neighbours: 4, CA: 0.6138552068473608
n_neighbours: 5, CA: 0.5935766365509589
n_neighbours: 6, CA: 0.6196405928039309
n_neighbours: 7, CA: 0.5939431764146457
n_neighbours: 8, CA: 0.6166686479632271
n_neighbours: 9, CA: 0.5974104453954667
n_neighbours: 10, CA: 0.6155095894753526
n_neighbours: 11, CA: 0.6006597717546363
n_neighbours: 12, CA: 0.6163615469963544
n_neighbours: 13, CA: 0.5986091298145506
n_neighbours: 14, CA: 0.6160544460294817
n_neighbours: 15, CA: 0.6023042478998257
n_neighbours: 16, CA: 0.6147963227135838
n_neighbours: 17, CA: 0.6047412426692027
n_neighbours: 18, CA: 0.6166785544460295
n_neighbours: 19, CA: 0.605553574258995
n_neighbours: 20, CA: 0.6176989221746712
n_neighbours: 21, CA: 0.6088821524805833
n_neighbours: 22, CA: 0.6178475194167063
n_neighbours: 23, CA: 0.6103086860041211
n_neighbours: 24, CA: 0.6256934537961643
n_neighbours: 25, CA: 0.61

In [27]:
n_neighbours = 24

classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=n_neighbours, p=np.inf)
classifier.fit(X_train_preproc, y_train)

y_hat = classifier.predict(X_test_preproc)

total_diff = np.sum(np.abs(y_test - y_hat) > 0)
ca = 1.0 - total_diff / n_test

print(f'n_neighbours: {n_neighbours}, CA: {ca}')

n_neighbours: 24, CA: 0.6256934537961643


In [28]:
# Which nodes can I detect accurately?

import json
import wntr
import src.analytics.analytics as analytics

network_fname = '../../data/epanet_networks/Braila_V2022_2_2.inp'
water_network_model = wntr.network.WaterNetworkModel(network_fname)
network_graph = water_network_model.get_graph()

nodeN_to_node_id_map = {nodeN: node_id[5:] for node_id, nodeN in enc_node_dict.items()}

# precompute the distances
node_dist_map = {}
for node_id in network_graph.nodes():
    path_len_vec, node_id_vec = analytics.undir_dist_from_set_vec(network_graph, [node_id])
    node_dist_map[node_id] = {
        'path_len_vec': path_len_vec,
        'node_id_vec': node_id_vec
    }

stats_map = {}
for valN, y_val_int64 in enumerate(y_test):
    y_val_str = nodeN_to_node_id_map[y_val_int64]
    y_hat_val = y_hat[valN]
    is_undetectable = 0
    
    if y_val_str not in stats_map:
        stats_map[y_val_str] = {'n_hits': 0, 'n_misses': 0, 'is_undetectable': is_undetectable, 'misclassify_id_map': {}, 'misclassify_dist_map': {}}
        
    if y_val_int64 == y_hat_val:
        stats_map[y_val_str]['n_hits'] += 1
    if y_val_int64 != y_hat_val:
        stats_map[y_val_str]['n_misses'] += 1
        
        y_hat_val_str = nodeN_to_node_id_map[y_hat_val]
        
        if y_hat_val_str not in stats_map[y_val_str]['misclassify_id_map']:
            stats_map[y_val_str]['misclassify_id_map'][y_hat_val_str] = 0
        stats_map[y_val_str]['misclassify_id_map'][y_hat_val_str] += 1
        
        # get the distance between y and y_hat
        node_dist_props = node_dist_map[y_val_str]
        path_len_vec = node_dist_props['path_len_vec']
        node_id_vec = node_dist_props['node_id_vec']
        
        misclassify_dist = -1
        for nodeN, node_id in enumerate(node_id_vec):
            if node_id == y_hat_val_str:
                misclassify_dist = path_len_vec[nodeN]
        assert misclassify_dist >= 0
        
        misclassify_dist = min(10000, misclassify_dist)
        
        #misclassify_dist = str(path_len_vec[node_id_vec.index(y_hat_val_str)])
        
        if misclassify_dist not in stats_map[y_val_str]['misclassify_dist_map']:
            stats_map[y_val_str]['misclassify_dist_map'][misclassify_dist] = 0
        stats_map[y_val_str]['misclassify_dist_map'][misclassify_dist] += 1

for node_id, node_stats in stats_map.items():
    total_samples = node_stats['n_hits'] + node_stats['n_misses']
    node_stats['hit_perc'] = float(node_stats['n_hits']) / total_samples
    
    misclassify_dist_map = node_stats['misclassify_dist_map']
    total_misclassify_count = 0
    mean_misclassify_dist = 0.0
    max_misclassify_dist = 0.0
    for misclassify_dist, misclassify_count in misclassify_dist_map.items():
        total_misclassify_count += misclassify_count
        mean_misclassify_dist += misclassify_dist*misclassify_count
        max_misclassify_dist = max(max_misclassify_dist, misclassify_dist)
    mean_misclassify_dist /= total_samples
        
    node_stats['mean_misclassify_dist'] = mean_misclassify_dist
    node_stats['max_misclassify_dist'] = max_misclassify_dist
    
with open(os.path.join(base_dir, 'model_stats.json'), 'w') as f_out:
    json_str = json.dumps(stats_map)
    f_out.write(json_str)
    

/home/lstopar/workspace/JSI/projects/naiades/braila-approximate-leakage/src/analytics/analytics.py:38: RuntimeWarning: divide by zero encountered in reciprocal
  np.reciprocal(reached_indicator_vec, out=tmp_n)


In [31]:
import pickle as pkl

# store the model to disk
with open(os.path.join(base_dir, 'data/models/model-knn-v2.pkl'), 'wb') as f_out:
    pkl.dump(classifier, f_out)